In [ ]:
# Copyright 2025 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-deepmind/genai-processors/blob/main/examples/research/research.ipynb)

# Genai Processors Research Agent 🧠

This notebook provides:

*   Setup instructions (including API key).
*   Step-by-step execution of individual processors (`TopicGenerator`, `TopicResearcher`, `TopicVerbalizer`).
*   Demonstration of chaining processors.
*   Running the complete `ResearchAgent`.
*   An advanced example applying the research agent to inform multimodal image generation.

## 1. 🛠️ Setup

In [ ]:
!pip install genai_processors

### API Key

To use the GenAI model processors, you will need an API key. If you have not
done so already, obtain your API key from Google AI Studio, and import it as a
secret in Colab (recommended) or directly set it below.

In [ ]:
from google.colab import userdata

API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
# @title Import modules
from typing import AsyncIterable

from genai_processors import content_api
from genai_processors import processor
from genai_processors import streams
from genai_processors.core import genai_model
from genai_processors.core import preamble
from genai_processors.examples import research
from google.genai import types as genai_types
from IPython.display import Markdown, display

ProcessorPart = processor.ProcessorPart


def render_part(part: ProcessorPart) -> None:
  if part.substream_name == 'status':
    display(Markdown(f'--- \n *Status*: {part.text}'))
  else:
    try:
      display(Markdown(part.text))
    except Exception:
      display(Markdown(f' {part.text} '))

In [ ]:
# @title Prompt
# @markdown Enter the prompt that will be used in the example
USER_PROMPT = "Research the best things about owning dalmatians!"  # @param { "type": "string" }

## 2. 🏗 Processors

The `ResearchAgent` is built on top of the Genai Processor library, leveraging its concurrency, chaining operations, and foundational tools for handling different data formats.

### `TopicGenerator`

The `TopicGenerator` processor generates a list of research topics based on the user's input content.

In [ ]:
p_generator = research.TopicGenerator(api_key=API_KEY)

topic_parts = []
input_stream = streams.stream_content([ProcessorPart(USER_PROMPT)])
async for content_part in p_generator(input_stream):
  if content_part.mimetype == 'application/json; type=Topic':
    topic_parts.append(content_part)
  else:
    render_part(content_part)

### `TopicResearcher`

The `TopicResearcher` processor researches specific `Topic` objects (produced by the `TopicGenerator`) and returns `ProcessorPart` objects containing `TopicResearch` in JSON format.

In [ ]:
p_researcher = research.TopicResearcher(api_key=API_KEY)

input_stream = streams.stream_content(topic_parts)
topic_research_parts = []
async for content_part in p_researcher.to_processor()(input_stream):
  if content_part.mimetype == 'application/json; type=Topic':
    topic_research_parts.append(content_part)
  else:
    render_part(content_part)

### `TopicVerbalizer`

The `TopicVerbalizer` processor converts `TopicResearch` parts into human-readable research text.

In [ ]:
p_verbalizer = research.TopicVerbalizer()

input_stream = streams.stream_content(topic_research_parts)
topic_verbalizer_parts = []
async for content_part in p_verbalizer.to_processor()(input_stream):
  render_part(content_part)
  topic_verbalizer_parts.append(content_part)

### Chaining the `TopicGenerator` and `TopicResearcher`


We can chain our processors together to seamlessly generate `Topic` objects for futher processing.

In [ ]:
topics = []

pipeline = p_generator + p_researcher

input_stream = streams.stream_content([ProcessorPart(USER_PROMPT)])
async for content_part in pipeline(input_stream):
  if content_part.mimetype == 'application/json; type=Topic':
    topics.append(content_part.get_dataclass(research.interfaces.Topic))
  else:
    render_part(content_part)

print(f'Researched {len(topics)} topics!')

## 3. 🤖 Agent

Now that we have all our building blocks, we can chain them together within our agent, resulting in a seamless flow of content.

In [ ]:
input_stream = streams.stream_content([ProcessorPart(USER_PROMPT)])

output_parts = []
async for content_part in research.ResearchAgent(api_key=API_KEY)(input_stream):
  if content_part.substream_name != 'status':
    output_parts.append(content_part)
  render_part(content_part)

In [ ]:
render_part(ProcessorPart(f"""# Final synthesized research

{content_api.as_text(output_parts)}"""))

## 4. 🖼️ Multimodal Agent Example

We can use reuse our research agent in different contexts.

For this example, imagine we are creating an image and want to ensure it is realistic. We can use core processors and our research agent to provide the image generator with useful tips.

In [ ]:
class ResearchedImageGenerator(processor.PartProcessor):

  def __init__(self, research_agent: research.ResearchAgent):
    research_suffix = preamble.Suffix(
        content=[
            ProcessorPart(
                'Please tailor your research so it can be used to provide an'
                " accurate image based on the user's content"
            )
        ]
    )
    self._research_pipeline = research_suffix + research_agent
    self._img_gen_model = genai_model.GenaiModel(
        api_key=API_KEY,
        model_name='gemini-2.0-flash-exp-image-generation',
        generate_content_config=genai_types.GenerateContentConfig(
            response_modalities=['Text', 'Image']
        ),
    )

  async def call(
      self, content_part: content_api.ProcessorPart
  ) -> AsyncIterable[content_api.ProcessorPart]:
    if not content_api.is_text(content_part.mimetype):
      raise ValueError('ResearchedImageGenerator expects text content')
      return

    user_prompt = content_part.text
    research_content = []

    research_input_stream = streams.stream_content([ProcessorPart(user_prompt)])

    async for research_part in self._research_pipeline(research_input_stream):
      yield research_part
      research_content.append(research_part)

    yield processor.status('Creating image...')
    img_gen_content = [
        ProcessorPart(
            "You are an expert at creating images based on a user's prompt and"
            ' using research provided to you'
        ),
        ProcessorPart(f'User prompt: {user_prompt}'),
        ProcessorPart(f'Research: {content_api.as_text(research_content)}'),
        ProcessorPart(
            f'Produce a high quality image, as well as an explanation of how'
            f' you used the research to inform your image.'
        ),
        ProcessorPart(f'Your image: '),
    ]

    img_gen_stream = processor.stream_content(img_gen_content)
    async for img_gen_part in self._img_gen_model(img_gen_stream):
      if content_api.is_image(img_gen_part.mimetype):
        yield processor.status('Generated image using research!')
      yield img_gen_part
    yield processor.status('Done!')

In [ ]:
image_generator = ResearchedImageGenerator(
    research.ResearchAgent(api_key=API_KEY)
)

In [ ]:
user_prompt = 'Create an image of two dalmatians, one with brown spots & one with black spots, frolicking in Crystal Palace park.'  # @param { "type": "string" }

In [ ]:
img_gen_results = []

async for content_part in image_generator(ProcessorPart(user_prompt)):
  if content_api.is_text(content_part.mimetype):
    render_part(content_part)
  elif content_part.pil_image:
    display(content_part.pil_image)
    img_gen_results.append(content_part)